In [12]:
from dotenv import load_dotenv

# Load API KEY information
load_dotenv(override=True)

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate


In [13]:
#Parameters and ChatMistral object creation


# Create the ChatMistralAI object
llm = ChatMistralAI(
    temperature=1,  # Low temperature for more focused responses
    model="mistral-small-latest", 
)

#If we want to understand pictures, we should use this model : "pixtral-12b-2409"

# Loading data

In [14]:
# Step 1: Load Documents
loader = PyMuPDFLoader("Data/Atlas.pdf")
docs = loader.load()
print(f"Number of pages in the document: {len(docs)}")



Number of pages in the document: 308


In [15]:
# Step 2: Split Documents
custom_separators = [
    "\n \n",        # paragraphs
    "\n",         # lines
    ". ",         # sentence-ish boundary
    "; ",         # clause boundary
    ", ",         # phrase boundary
    " ",          # words
    ""            # fallback: characters
]
text_splitter = RecursiveCharacterTextSplitter(separators = custom_separators, chunk_size=500, chunk_overlap=50)      #Paramètre à modifier par la suite pour de meilleur performance
split_documents = text_splitter.split_documents(docs)
print(f"Number of split chunks: {len(split_documents)}")

Number of split chunks: 696


In [16]:
# Step 3: Generate Embeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

c:\Users\Comor\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Step 4: Create and Save the Database
# Create a vector store
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
print("Vector store created successfully!")

Vector store created successfully!


In [18]:
# Step 5: Create Retriever
# Search and retrieve information contained in the documents
retriever = vectorstore.as_retriever()

In [19]:
# Step 6: Create Prompt


prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [20]:
# Step 7: Setup LLM
llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

In [21]:
# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
#Exemple

# Run Chain
# Input a query about the document and print the response
question = "Qui a gagné le ballon d'or en 2009"
response = chain.invoke(question)
print(response)
print("-----")
question = "Quel pays a légalisé l'avortement en premier dans le monde ?"
response = chain.invoke(question)
print(response)


Je ne sais pas.
-----
D'après le contexte fourni, l'Islande est le premier pays à avoir légalisé l'avortement, dès 1934.


# Prompt engineering

Here we setup a basic template for our prompt engineering.
In our case, the LLM will be a specialist in geography, in secondary school.

The student will interact with the LLM in two different ways :
    -He can ask any type of question about any topic in the course.
    -He can ask to have his knowledge tested (he will then receive a score on his answer and a feedback)

In [23]:
# Persona prompt, specific for when the student has a question about a specific part of the course

persona_template = (
    "Act as a supportive but rigorous geography teacher.\n" \
    "Your tone should be constructive, specific, and pedagogical.\n" \
        """Tu es un professeur de géographie avec 20 ans d'expérience, et ton but est de répondre aux questions d'un élève en difficulté.
            Tu es encourageant, mais tout de fois rigoureux quant à la précision de tes réponses.

    CONTRAINTES:
    1. Utilise UNIQUEMENT le contexte fourni qui vient d .
    2. Cite chaque fait avec la page sous forme [Page X].
    3. Ne fabrique rien.

    Format attendu:
    Réponse concise en français.
    CITES: Page: X,Y,... (liste unique de pages utilisées)

    Question: {question}

    Contexte:
    {context}
    """
)

scores = """
- Pertinence : Est-ce que l'étudiant répond bien à la question posé et non pas à autre chose  /30;
- Faits non correctes: Est-ce qu'il y'a des faits qui ne sont pas correctes dans la réponse  /30;
- Faits manquants : Est-ce que tous les faits attendus sont bien présent dans la réponse  /30;
- Stucture : Est-ce que la réponse est bien stucturée /10;
"""

test_template = (    f"Act as a supportive but rigorous history teacher.\n"
    "Your goal is to generate a question based on the course."             
    "The student gives you an answer and your goal is to evaluate it.\n"
    "Assignment requirement: {task_description}\n"
    "Grading rubric: {grading_rubric}\n"
    "Return ONLY a JSON object with these keys:\n"
    "- Section: the general theme of the question\n"
    "- Question: the question you asked the student\n"
    "- Answer: The answer the student gave\n"
    "- grade: number (0-100), must equal sum of all scores\n"
    "- scores: f{scores} \n"
    "- advice: array of short, actionable improvement suggestions\n"
    "Constraints: grade MUST equal Pertinence+Faits non correctes + Faits manquants + Structure. No extra text outside the JSON.\n\n"
    )

# Data base

In [9]:
import sqlite3
import json

# -------------------------------
# 1. Create the SQLite database
# -------------------------------
conn = sqlite3.connect('student_results.db')  # This creates a file on disk
cursor = conn.cursor()

# Create table for storing answers and grading
cursor.execute('''
CREATE TABLE IF NOT EXISTS student_results (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT,
    question TEXT,
    answer TEXT,
    grade REAL,
    scores TEXT,        -- we'll store JSON as a string
    advice TEXT
)
''')
conn.commit()
print("Database 'student_results.db' ready")

# -------------------------------
# 2. Function to save a result
# -------------------------------
def save_result(student_name, question, answer, grading_json):
    """
    grading_json: dictionary returned by LLM in test mode
    """
    # Convert the 'scores' dict to a JSON string
    scores_str = json.dumps(grading_json.get("scores", {}), ensure_ascii=False)

    cursor.execute('''
        INSERT INTO student_results (student_name, question, answer, grade, scores, advice)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        student_name,
        question,
        answer,
        grading_json.get("grade", 0),
        scores_str,
        grading_json.get("advice", "")
    ))
    conn.commit()
    print(f"Result for {student_name} saved successfully!")

# -------------------------------
# 3. Example usage
# -------------------------------

# Suppose we have a grading result from the LLM (already parsed as JSON)
example_grading_json = {
    "Section": "Histoire",
    "Question": "Quels furent les principaux événements qui ont marqué le début de la Seconde Guerre mondiale en Europe ?",
    "Answer": "L'invasion de la Pologne par l'Allemagne, et le fait qu'il y'avait une crise économique assez forte",
    "grade": 40,
    "scores": {
        "Pertinence": 20,
        "Faits non correctes": 20,
        "Faits manquants": 40,
        "Structure": 10
    },
    "advice": "La réponse mentionne correctement l'invasion de la Pologne, qui est un événement clé. Cependant, la crise économique, bien que pertinente, n'est pas un événement marquant du début de la guerre..."
}

#save_result("Edin", example_grading_json["Question"], example_grading_json["Answer"], example_grading_json)




Database 'student_results.db' ready


# Vector embeddings

In [1]:
# ============================================================
# FULL WORKING VERSION — RAG + PERSONA CHAIN + GRADING CHAIN
# ============================================================

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch


# ============================================================
# 1. LLM CONFIG
# ============================================================

llm = ChatMistralAI(
    model="mistral-small-latest",   # Vision model would be: pixtral-12b-2409
    temperature=1
)

# OPTIONAL: if you want vision, replace above with:
# llm = ChatMistralAI(model="pixtral-12b-2409", temperature=1)



# ============================================================
# 2. LOAD DOCUMENTS (PDF)
# ============================================================

loader = PyMuPDFLoader("Data/Atlas.pdf")
docs = loader.load()



# ============================================================
# 3. SPLIT DOCUMENTS
# ============================================================

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=[
        "\nCHAPITRE", "\n##", "\n###", "\nSection", "\n\n", "\n", ".", " ", ""
    ]
)

split_documents = text_splitter.split_documents(docs)



# ============================================================
# 4. EMBEDDINGS + VECTORSTORE + RETRIEVER
# ============================================================

embeddings = MistralAIEmbeddings(model="mistral-embed")

vectorstore = FAISS.from_documents(split_documents, embeddings)

retriever = vectorstore.as_retriever()

# Save to disk
vectorstore.save_local("faiss_index")
print("Vectorstore saved to 'faiss_index/'")

c:\Users\Comor\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vectorstore saved to 'faiss_index/'


# Interface

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_mistralai import MistralAIEmbeddings

# Initialize embeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

# Load the vectorstore (allow unsafe deserialization since you created it)
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()
print("Vectorstore loaded successfully")



# ============================================================
# 5. PERSONA CHAIN (Teacher mode + RAG)
# ============================================================

persona_template = ChatPromptTemplate.from_messages([
    ("system",
     "Tu es un professeur bienveillant. Explique simplement mais sans infantiliser. "
     "Appuie-toi uniquement sur le contexte fourni."),
    ("human",
     "Question: {question}\n\n"
     "Contexte issu des documents:\n{context}")
])

persona_chain = (
    persona_template
    | llm
)



# ============================================================
# 6. GRADING CHAIN (Automatic evaluation)
# ============================================================

test_template = ChatPromptTemplate.from_messages([
    ("system",
     "Tu es un correcteur automatique. Évalue la réponse de l'élève selon les critères fournis."),
    ("human",
     #"Instruction donnée à l'élève : {task_description}\n"
     "Barème : {grading_rubric}\n"
     "Question : {question}\n"
     "Réponse de l'élève : {answer}\n\n"
     "Donne une note sur 20 + justification.")
])

test_chain = (
    test_template
    | llm
)



# ============================================================
# 7. RAG WRAPPER — run retrieval only if needed
# ============================================================

def rag_logic(inputs):
    query = inputs.get("question", "")
    if not query:
        inputs["context"] = ""
        return inputs

    docs = retriever.invoke(query)
    ctx = "\n\n".join(doc.page_content for doc in docs)

    inputs["context"] = ctx
    return inputs


rag_chain = RunnableLambda(rag_logic)



# ============================================================
# 8. ROUTING — decide whether to use Persona or Test chain
# ============================================================

def route(inputs):
    """
    If 'answer' is provided → grading mode.
    Otherwise → persona/teacher mode.
    """
    return "answer" not in inputs


conditional_chain = RunnableBranch(
    # condition → persona mode
    (
        lambda inputs: route(inputs),
        rag_chain | persona_chain
    ),
    # fallback → grading mode
    test_template | llm
)

def generate_test_question(criteria):
    """
    Generate a test question using document context (RAG) and student instructions.
    """
    # Step 1: Retrieve relevant context using RAG
    # Use invoke() method to retrieve documents
    docs = retriever.invoke(criteria)

    context_text = "\n\n".join([doc.page_content for doc in docs])

    # Step 2: Prompt for question generation using the retrieved context
    question_gen_prompt = ChatPromptTemplate.from_messages([
        ("system",
         "Tu es un professeur bienveillant et rigoureux. "
         "À partir du contexte fourni, génère une question pertinente pour un élève."),
        ("human",
         "Instructions : {criteria}\n\nContexte : {context}")
    ])

    question_gen_chain = question_gen_prompt | llm

    generated_question = question_gen_chain.invoke({
        "criteria": criteria,
        "context": context_text
    }).content.strip()

    return generated_question



# ============================================================
# 9. MAIN ENTRYPOINT
# ============================================================

def respond(inputs):
    return conditional_chain.invoke(inputs)



# ============================================================
# 10. INTERACTIVE TESTING WITH input()
# ============================================================

if __name__ == "__main__":

    print("=== Tutor System Running ===")
    print("Ask a question to the teacher, or type 'test' to grade an answer.\n")

    mode = input("Mode (teach/test): ").strip().lower()

    if mode == "teach":
        question = input("Your question: ")
        result = respond({"question": question})
        print("\n--- Teacher answer ---")
        print(result)

    elif mode == "test":
        # Étudiant fournit uniquement le barème et sa réponse
        rubric = input("Barème : ")

        # --- Etape 1 : Generate question using RAG context ---
        generated_question = generate_test_question(rubric)
        print("\n--- Question générée automatiquement ---")
        print(generated_question)

        # --- Étape 2 : l'élève fournit sa réponse ---
        answer = input("\nRéponse de l'élève : ")

        # --- Étape 3 : prompt de correction (JSON output) ---
        scores_text = (
            "- Pertinence : ... /30;\n"
            "- Faits non correctes : ... /30;\n"
            "- Faits manquants : ... /30;\n"
            "- Structure : ... /10;"
        )

        test_prompt_template = ChatPromptTemplate.from_messages([
            ("system",
            "Act as a supportive but rigorous history teacher.\n"
            "Your goal is to evaluate the student's answer and return ONLY a JSON object."),
            ("human",
            "Grading rubric: {grading_rubric}\n"
            "Question: {question}\n"
            "Answer: {answer}\n"
            "Scores template: {scores_text}\n"
            "Constraints: grade MUST equal sum of all scores.\n"
            "Return a JSON object with keys:\n"
            "- Section\n"
            "- Question\n"
            "- Answer\n"
            "- grade (0-100)\n"
            "- scores\n"
            "- advice\n"
            "No extra text or Markdown, ONLY JSON.")
        ])

        test_chain = test_prompt_template | llm

        grading_result = test_chain.invoke({
            "grading_rubric": rubric,
            "question": generated_question,
            "answer": answer,
            "scores_text": scores_text
        })

        # --- Étape 4 : transformer la string en dictionnaire Python ---
        import json

        raw_output = grading_result.content.strip()

        # Retirer les ```json ou ``` éventuels
        if raw_output.startswith("```"):
            raw_output = "\n".join(raw_output.split("\n")[1:-1])

        try:
            grading_json = json.loads(raw_output)
        except json.JSONDecodeError:
            print("Erreur : le LLM n'a pas retourné un JSON valide")
            grading_json = None

        # --- Étape 5 : afficher le résultat ---
        if grading_json:
            print("\n--- JSON Grading Result ---")
            print(grading_json)
            print("\nNote :", grading_json["grade"])
            print("Conseils :", grading_json["advice"])
        #save_result("Edin", grading_json["Question"], grading_json["Answer"], grading_json)



    else:
        print("Unknown mode.")



Vectorstore loaded successfully
=== Tutor System Running ===
Ask a question to the teacher, or type 'test' to grade an answer.


--- Question générée automatiquement ---
Question pertinente pour un élève :

"En vous basant sur le contexte fourni, expliquez comment l'Angleterre a utilisé son influence diplomatique pour façonner la carte politique de l'Europe après le Congrès de Vienne en 1815. Quels étaient ses objectifs et quels moyens a-t-elle employés pour les atteindre ?"

Cette question encourage l'élève à analyser les actions de l'Angleterre dans le contexte post-napoléonien, à comprendre ses motivations stratégiques et à évaluer l'impact de ses politiques sur les autres nations européennes.

--- JSON Grading Result ---
{'Section': 'Histoire', 'Question': "En vous basant sur le contexte fourni, expliquez comment l'Angleterre a utilisé son influence diplomatique pour façonner la carte politique de l'Europe après le Congrès de Vienne en 1815. Quels étaient ses objectifs et quels moy

In [11]:
# -------------------------------
# 4. Query the database
# -------------------------------
cursor.execute("SELECT * FROM student_results")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close connection when done
conn.close()

(1, 'Edin', 'Quelles étaient les principales causes de la Première Guerre mondiale et comment ont-elles conduit à son déclenchement en 1914 ?', "L'assassination de l'archeduc François ferdinant", 20.0, '{"Pertinence": 10, "Faits non correctes": 20, "Faits manquants": 20, "Structure": 0}', "Votre réponse est incomplète et manque de structure. Vous avez mentionné un événement important, mais vous devez inclure d'autres causes majeures comme le nationalisme, l'impérialisme, le militarisme et les alliances. De plus, vous devez expliquer comment ces facteurs ont conduit à la guerre en 1914. Travaillez sur la clarté et l'organisation de votre réponse.")


In [8]:
print(grading_json['scores'])


{'Pertinence': 0, 'Faits non correctes': 30, 'Faits manquants': 30, 'Structure': 10}
